# Proximal Policy Optimization (PPO) Tutorial

## 1. Environment Preparation




### 1.1 Download Packages for BipedalWalker-v3

In [1]:
!pip install gym pyvirtualdisplay > /dev/null 2>&1
!apt-get install -y xvfb python-opengl ffmpeg > /dev/null 2>&1
!pip install colabgymrender==1.0.2
!pip install box2d-py
!pip install 'gym[Box2D]'

  Created wheel for colabgymrender: filename=colabgymrender-1.0.2-cp37-none-any.whl size=2446 sha256=dd65d054c08d16709375913a090748a5dd0c5fae8941a6642a58637eea6e9e58
  Stored in directory: /root/.cache/pip/wheels/7e/75/39/ba42a666bfb04ef568d25f43e5e963bb30c275446271085240
Successfully built colabgymrender
     |████████████████████████████████| 450kB 5.0MB/s 


### 1.2 Mount Drive and Set Project Path

In [2]:
from google.colab import drive
drive.mount('/content/drive')

import sys
project_root = '/content/drive/My Drive/ppo_tutorial/'
sys.path.append(project_root)

Mounted at /content/drive


### 1.3 Test the BipedalWalker-v3 Environment 

In [3]:
import os
import gym
import torch
import torch.nn as nn
import numpy as np
from colabgymrender.recorder import Recorder

env = gym.make('BipedalWalker-v3')
s_dim = env.observation_space.shape[0]
a_dim = env.action_space.shape[0]
print(s_dim)
print(a_dim)

Imageio: 'ffmpeg-linux64-v3.3.1' was not found on your computer; downloading it now.
Try 1. Download from https://github.com/imageio/imageio-binaries/raw/master/ffmpeg/ffmpeg-linux64-v3.3.1 (43.8 MB)
Downloading: 8192/45929032 bytes (0.0%)2080768/45929032 bytes (4.5%)5349376/45929032 bytes (11.6%)8429568/45929032 bytes (18.4%)11493376/45929032 bytes (25.0%)14680064/45929032 bytes (32.0%)17645568/45929032 bytes (38.4%)20676608/45929032 bytes (45.0%)23830528/45929032 bytes (51.9%)26877952/45929032 bytes (58.5%)29843456/45929032 bytes (65.0%)32915456/45929032 bytes (71.7%)35823616/45929032 bytes (78.0%)

## 2. Policy Network & Value Network Construction

### 2.1 Diagonal Gaussian Distribution Module 

In [4]:
#AddBias module
class AddBias(nn.Module):
    def __init__(self, bias):
        super(AddBias, self).__init__()
        self._bias = nn.Parameter(bias.unsqueeze(1))
    
    def forward(self, x):
        bias = self._bias.t().view(1, -1)
        return x + bias

#Gaussian distribution with given mean & std.
class FixedNormal(torch.distributions.Normal):
    def log_probs(self, x):
        return super().log_prob(x).sum(-1)
    
    def entropy(self):
        return super().entropy().sum(-1)

    def mode(self):
        return self.mean

#Diagonal Gaussian module
class DiagGaussian(nn.Module):
    def __init__(self, inp_dim, out_dim):
        super(DiagGaussian, self).__init__()
        self.fc_mean = nn.Linear(inp_dim, out_dim)
        self.b_logstd = AddBias(torch.zeros(out_dim))
    
    def forward(self, x):
        mean = self.fc_mean(x)
        logstd = self.b_logstd(torch.zeros_like(mean))
        return FixedNormal(mean, logstd.exp())

### 2.2 Policy Network & Value Network Module

In [5]:
#Policy Network
class PolicyNet(nn.Module):
    #Constructor
    def __init__(self, s_dim, a_dim):
        super(PolicyNet, self).__init__()
        self.main = nn.Sequential(
            nn.Linear(s_dim, 128),
            nn.ReLU(),
            nn.Linear(128, 128),
            nn.ReLU()
        )
        self.dist = DiagGaussian(128, a_dim)
    
    #Forward pass
    def forward(self, state, deterministic=False):
        feature = self.main(state)
        dist = self.dist(feature)

        if deterministic:
            action = dist.mode()
        else:
            action = dist.sample()
        
        return action, dist.log_probs(action)
    
    #Choose an action (stochastically or deterministically)
    def choose_action(self, state, deterministic=False):
        feature = self.main(state)
        dist = self.dist(feature)

        if deterministic:
            return dist.mode()

        return dist.sample()
    
    #Evaluate a state-action pair (output log-prob. & entropy)
    def evaluate(self, state, action):
        feature = self.main(state)
        dist = self.dist(feature)
        return dist.log_probs(action), dist.entropy()

#Value Network
class ValueNet(nn.Module):
    #Constructor
    def __init__(self, s_dim):
        super(ValueNet, self).__init__()
        self.main = nn.Sequential(
            nn.Linear(s_dim, 128),
            nn.ReLU(),
            nn.Linear(128, 128),
            nn.ReLU(),
            nn.Linear(128, 1)
        )
    
    #Forward pass
    def forward(self, state):
        return self.main(state)[:, 0]

### 2.3 Create Policy Network & Value Network

In [6]:
policy_net = PolicyNet(s_dim, a_dim)
value_net = ValueNet(s_dim)
print(policy_net)
print(value_net)

PolicyNet(
  (main): Sequential(
    (0): Linear(in_features=24, out_features=128, bias=True)
    (1): ReLU()
    (2): Linear(in_features=128, out_features=128, bias=True)
    (3): ReLU()
  )
  (dist): DiagGaussian(
    (fc_mean): Linear(in_features=128, out_features=4, bias=True)
    (b_logstd): AddBias()
  )
)
ValueNet(
  (main): Sequential(
    (0): Linear(in_features=24, out_features=128, bias=True)
    (1): ReLU()
    (2): Linear(in_features=128, out_features=128, bias=True)
    (3): ReLU()
    (4): Linear(in_features=128, out_features=1, bias=True)
  )
)


## 3. Environment Runner Construction

### 3.1 EnvRunner Class

In [7]:
class EnvRunner:
    #Constructor
    def __init__(self, s_dim, a_dim, gamma=0.99, lamb=0.95, max_step=2048, device='cpu'):
        self.s_dim = s_dim
        self.a_dim = a_dim
        self.gamma = gamma
        self.lamb = lamb
        self.max_step = max_step
        self.device = device

        #Storages (state, action, value, reward, a_logp)
        self.mb_states = np.zeros((self.max_step, self.s_dim), dtype=np.float32)
        self.mb_actions = np.zeros((self.max_step, self.a_dim), dtype=np.float32)
        self.mb_values = np.zeros((self.max_step,), dtype=np.float32)
        self.mb_rewards = np.zeros((self.max_step,), dtype=np.float32)
        self.mb_a_logps = np.zeros((self.max_step,), dtype=np.float32)
    
    #Compute discounted return
    def compute_discounted_return(self, rewards, last_value):
        returns = np.zeros_like(rewards)
        n_step = len(rewards)

        for t in reversed(range(n_step)):
            if t == n_step - 1:
                returns[t] = rewards[t] + self.gamma * last_value
            else:
                returns[t] = rewards[t] + self.gamma * returns[t+1]

        return returns
    
    #Compute generalized advantage estimation (Optional)
    def compute_gae(self, rewards, values, last_value):
        advs = np.zeros_like(rewards)
        n_step = len(rewards)
        last_gae_lam = 0.0

        for t in reversed(range(n_step)):
            if t == n_step - 1:
                next_value = last_value
            else:
                next_value = values[t+1]

            delta = rewards[t] + self.gamma*next_value - values[t]
            advs[t] = last_gae_lam = delta + self.gamma*self.lamb*last_gae_lam

        return advs + values

    #Run an episode using the policy net & value net
    def run(self, env, policy_net, value_net):
        #Run an episode
        state = env.reset()   #Initial state
        episode_len = self.max_step

        for step in range(self.max_step):
            state_tensor = torch.tensor(np.expand_dims(state, axis=0), dtype=torch.float32, device=self.device)
            action, a_logp = policy_net(state_tensor)
            value = value_net(state_tensor)

            action = action.cpu().numpy()[0]
            a_logp = a_logp.cpu().numpy()
            value  = value.cpu().numpy()

            self.mb_states[step] = state
            self.mb_actions[step] = action
            self.mb_a_logps[step] = a_logp
            self.mb_values[step] = value

            state, reward, done, info = env.step(action)
            self.mb_rewards[step] = reward

            if done:
                episode_len = step + 1
                break
        
        #Compute returns
        last_value = value_net(
            torch.tensor(np.expand_dims(state, axis=0), dtype=torch.float32, device=self.device)
        ).cpu().numpy()

        mb_returns = self.compute_discounted_return(self.mb_rewards[:episode_len], last_value)
        '''
        mb_returns = self.compute_gae(
            self.mb_rewards[:episode_len], 
            self.mb_values[:episode_len],
            last_value
        )
        '''
        return self.mb_states[:episode_len], \
                self.mb_actions[:episode_len], \
                self.mb_a_logps[:episode_len], \
                self.mb_values[:episode_len], \
                mb_returns, \
                self.mb_rewards[:episode_len]

### 3.2 Create EnvRunner

In [8]:
runner = EnvRunner(s_dim, a_dim)

## 4. PPO Algorithm

### 4.1 PPO Class

In [9]:
class PPO:
    #Constructor
    def __init__(self, policy_net, value_net, lr=1e-4, max_grad_norm=0.5, ent_weight=0.01, clip_val=0.2, sample_n_epoch=4, sample_mb_size=64, device='cpu'):
        self.policy_net = policy_net
        self.value_net = value_net
        self.max_grad_norm = max_grad_norm
        self.ent_weight = ent_weight
        self.clip_val = clip_val
        self.sample_n_epoch = sample_n_epoch
        self.sample_mb_size = sample_mb_size
        self.device = device
        self.opt_polcy = torch.optim.Adam(policy_net.parameters(), lr)
        self.opt_value = torch.optim.Adam(value_net.parameters(), lr)
    
    #Train the policy net & value net using PPO
    def train(self, mb_states, mb_actions, mb_old_values, mb_advs, mb_returns, mb_old_a_logps):
        #Convert numpy array to tensor
        mb_states = torch.from_numpy(mb_states).to(self.device)
        mb_actions = torch.from_numpy(mb_actions).to(self.device)
        mb_old_values = torch.from_numpy(mb_old_values).to(self.device)
        mb_advs = torch.from_numpy(mb_advs).to(self.device)
        mb_returns = torch.from_numpy(mb_returns).to(self.device)
        mb_old_a_logps = torch.from_numpy(mb_old_a_logps).to(self.device)
        episode_length = len(mb_states)
        rand_idx = np.arange(episode_length)
        sample_n_mb = episode_length // self.sample_mb_size

        if sample_n_mb <= 0:
            sample_mb_size = episode_length
            sample_n_mb = 1
        else:
            sample_mb_size = self.sample_mb_size

        for i in range(self.sample_n_epoch):
            np.random.shuffle(rand_idx)

            for j in range(sample_n_mb):
                #Randomly sample a batch for training
                sample_idx = rand_idx[j*sample_mb_size : (j+1)*sample_mb_size]
                sample_states = mb_states[sample_idx]
                sample_actions = mb_actions[sample_idx]
                sample_old_values = mb_old_values[sample_idx]
                sample_advs = mb_advs[sample_idx]
                sample_returns = mb_returns[sample_idx]
                sample_old_a_logps = mb_old_a_logps[sample_idx]

                sample_a_logps, sample_ents = self.policy_net.evaluate(sample_states, sample_actions)
                sample_values = self.value_net(sample_states)
                ent = sample_ents.mean()

                #Compute value loss
                v_pred_clip = sample_old_values + torch.clamp(sample_values - sample_old_values, -self.clip_val, self.clip_val)
                v_loss1 = (sample_returns - sample_values).pow(2)
                v_loss2 = (sample_returns - v_pred_clip).pow(2)
                v_loss = torch.max(v_loss1, v_loss2).mean()

                #Compute policy gradient loss
                ratio = (sample_a_logps - sample_old_a_logps).exp()
                pg_loss1 = -sample_advs * ratio
                pg_loss2 = -sample_advs * torch.clamp(ratio, 1.0-self.clip_val, 1.0+self.clip_val)
                pg_loss = torch.max(pg_loss1, pg_loss2).mean() - self.ent_weight*ent

                #Train actor
                self.opt_polcy.zero_grad()
                pg_loss.backward()
                nn.utils.clip_grad_norm_(self.policy_net.parameters(), self.max_grad_norm)
                self.opt_polcy.step()

                #Train critic
                self.opt_value.zero_grad()
                v_loss.backward()
                nn.utils.clip_grad_norm_(self.value_net.parameters(), self.max_grad_norm)
                self.opt_value.step()

        return pg_loss.item(), v_loss.item(), ent.item()

### 4.2 Create PPO Agent

In [10]:
agent = PPO(policy_net, value_net)

## 5. Training and Testing Process

### 5.1 Play an Episode for Evaluation

In [11]:
def play(policy_net):
    render_env = Recorder(gym.make('BipedalWalker-v3'), project_root + '/video')

    with torch.no_grad():
        state = render_env.reset()
        total_reward = 0
        length = 0

        while True:
            #render_env.render()
            state_tensor = torch.tensor(np.expand_dims(state, axis=0), dtype=torch.float32, device='cpu')
            action = policy_net.choose_action(state_tensor, deterministic=True).cpu().numpy()
            state, reward, done, info = render_env.step(action[0])
            total_reward += reward
            length += 1

            if done:
                print("[Evaluation] Total reward = {:.6f}, length = {:d}".format(total_reward, length), flush=True)
                break
    render_env.play()
    render_env.close()

In [12]:
play(policy_net)

[Evaluation] Total reward = -91.896360, length = 107


 99%|█████████▉| 108/109 [00:00<00:00, 262.70it/s]


### 5.2 Train the Networks using PPO

In [13]:
def train(env, runner, policy_net, value_net, agent, max_episode=5000):
    mean_total_reward = 0
    mean_length = 0
    save_dir = project_root + '/save'

    if not os.path.exists(save_dir):
        os.mkdir(save_dir)

    for i in range(max_episode):
        #Run an episode to collect data
        with torch.no_grad():
            mb_states, mb_actions, mb_old_a_logps, mb_values, mb_returns, mb_rewards = runner.run(env, policy_net, value_net)
            mb_advs = mb_returns - mb_values
            mb_advs = (mb_advs - mb_advs.mean()) / (mb_advs.std() + 1e-6)
        
        #Train the model using the collected data
        pg_loss, v_loss, ent = agent.train(mb_states, mb_actions, mb_values, mb_advs, mb_returns, mb_old_a_logps)
        mean_total_reward += mb_rewards.sum()
        mean_length += len(mb_states)
        print("[Episode {:4d}] total reward = {:.6f}, length = {:d}".format(i, mb_rewards.sum(), len(mb_states)))

        #Show the current result & save the model
        if i % 200 == 0:
            print("\n[{:5d} / {:5d}]".format(i, max_episode))
            print("----------------------------------")
            print("actor loss = {:.6f}".format(pg_loss))
            print("critic loss = {:.6f}".format(v_loss))
            print("entropy = {:.6f}".format(ent))
            print("mean return = {:.6f}".format(mean_total_reward / 200))
            print("mean length = {:.2f}".format(mean_length / 200))
            print("\nSaving the model ... ", end="")
            torch.save({
                "it": i,
                "PolicyNet": policy_net.state_dict(),
                "ValueNet": value_net.state_dict()
            }, os.path.join(save_dir, "model.pt"))
            print("Done.")
            print()
            play(policy_net)
            mean_total_reward = 0
            mean_length = 0

In [14]:
train(env, runner, policy_net, value_net, agent)
env.close()

[Episode    0] total reward = -100.726105, length = 58

[    0 /  5000]
----------------------------------
actor loss = -0.064683
critic loss = 5872.854004
entropy = 5.676354
mean return = -0.503631
mean length = 0.29

Saving the model ... Done.

[Evaluation] Total reward = -92.398783, length = 106


100%|██████████| 108/108 [00:00<00:00, 267.36it/s]


[Episode    1] total reward = -204.640762, length = 1031
[Episode    2] total reward = -101.502663, length = 71
[Episode    3] total reward = -120.150291, length = 102
[Episode    4] total reward = -108.430710, length = 1600
[Episode    5] total reward = -101.028984, length = 67
[Episode    6] total reward = -130.418747, length = 128
[Episode    7] total reward = -103.032234, length = 82
[Episode    8] total reward = -102.464142, length = 66
[Episode    9] total reward = -102.171600, length = 56
[Episode   10] total reward = -98.492165, length = 90
[Episode   11] total reward = -114.213806, length = 52
[Episode   12] total reward = -103.883438, length = 58
[Episode   13] total reward = -101.103027, length = 74
[Episode   14] total reward = -103.736259, length = 82
[Episode   15] total reward = -127.204727, length = 104
[Episode   16] total reward = -100.017090, length = 86
[Episode   17] total reward = -98.075729, length = 1600
[Episode   18] total reward = -116.049957, length = 63
[Ep

 99%|█████████▉| 114/115 [00:00<00:00, 257.48it/s]


[Episode  201] total reward = -118.356674, length = 64
[Episode  202] total reward = -111.698143, length = 62
[Episode  203] total reward = -103.272301, length = 56
[Episode  204] total reward = -108.632858, length = 55
[Episode  205] total reward = -99.083115, length = 110
[Episode  206] total reward = -109.133118, length = 51
[Episode  207] total reward = -112.712479, length = 85
[Episode  208] total reward = -117.089157, length = 159
[Episode  209] total reward = -142.641113, length = 1066
[Episode  210] total reward = -50.648354, length = 1600
[Episode  211] total reward = -106.942024, length = 71
[Episode  212] total reward = -64.845367, length = 1600
[Episode  213] total reward = -106.015991, length = 52
[Episode  214] total reward = -114.473694, length = 146
[Episode  215] total reward = -108.575813, length = 87
[Episode  216] total reward = -112.226074, length = 82
[Episode  217] total reward = -53.641434, length = 1600
[Episode  218] total reward = -161.816711, length = 1366
[

...

[Episode 4401] total reward = -119.712265, length = 44
[Episode 4402] total reward = 202.810364, length = 1599
[Episode 4403] total reward = 94.124634, length = 1366
[Episode 4404] total reward = -124.372292, length = 81
[Episode 4405] total reward = 225.030975, length = 1374
[Episode 4406] total reward = 98.798874, length = 1328
[Episode 4407] total reward = 229.467102, length = 1328
[Episode 4408] total reward = -115.548027, length = 52
[Episode 4409] total reward = 230.629868, length = 1327
[Episode 4410] total reward = 221.039108, length = 1421
[Episode 4411] total reward = 234.790283, length = 1281
[Episode 4412] total reward = 235.156784, length = 1256
[Episode 4413] total reward = 236.724792, length = 1257
[Episode 4414] total reward = -122.852211, length = 72
[Episode 4415] total reward = -130.432693, length = 110
[Episode 4416] total reward = -115.369087, length = 55
[Episode 4417] total reward = -116.413803, length = 97
[Episode 4418] total reward = -115.225372, length = 54
[

100%|██████████| 1000/1000 [00:03<00:00, 253.86it/s]


[Episode 4601] total reward = -52.519630, length = 338
[Episode 4602] total reward = 238.579071, length = 1226
[Episode 4603] total reward = 241.933945, length = 1184
[Episode 4604] total reward = 248.080139, length = 1112
[Episode 4605] total reward = 39.998238, length = 814
[Episode 4606] total reward = -132.300980, length = 119
[Episode 4607] total reward = -75.890404, length = 237
[Episode 4608] total reward = 235.947006, length = 1244
[Episode 4609] total reward = -131.136017, length = 122
[Episode 4610] total reward = 36.222031, length = 662
[Episode 4611] total reward = 242.152069, length = 1188
[Episode 4612] total reward = 236.373260, length = 1255
[Episode 4613] total reward = 240.390076, length = 1210
[Episode 4614] total reward = 2.209774, length = 639
[Episode 4615] total reward = 76.010658, length = 837
[Episode 4616] total reward = 243.239944, length = 1171
[Episode 4617] total reward = -122.764099, length = 126
[Episode 4618] total reward = -113.578445, length = 269
[Ep

100%|██████████| 690/690 [00:02<00:00, 253.91it/s]


[Episode 4801] total reward = 249.623474, length = 1085
[Episode 4802] total reward = 253.588013, length = 1068
[Episode 4803] total reward = 241.963821, length = 1161
[Episode 4804] total reward = 253.905334, length = 1040
[Episode 4805] total reward = -112.867630, length = 43
[Episode 4806] total reward = -19.134758, length = 673
[Episode 4807] total reward = 246.387024, length = 1138
[Episode 4808] total reward = -109.802513, length = 57
[Episode 4809] total reward = 244.366425, length = 1158
[Episode 4810] total reward = 243.761200, length = 1132
[Episode 4811] total reward = 249.348709, length = 1092
[Episode 4812] total reward = 247.852509, length = 1111
[Episode 4813] total reward = 246.663574, length = 1121
[Episode 4814] total reward = 250.244934, length = 1102
[Episode 4815] total reward = 240.546204, length = 1193
[Episode 4816] total reward = 248.437668, length = 1115
[Episode 4817] total reward = 246.647110, length = 1141
[Episode 4818] total reward = 255.978745, length = 

### 5.3 Load the Model and Play

In [15]:
save_dir = project_root + '/save'
model_path = os.path.join(save_dir, "model.pt")

if os.path.exists(model_path):
    print("Loading the model ... ", end="")
    checkpoint = torch.load(model_path)
    policy_net.load_state_dict(checkpoint["PolicyNet"])
    print("Done.")
else:
    print('ERROR: No model saved')

play(policy_net)

Loading the model ... Done.
[Evaluation] Total reward = 263.157728, length = 952


100%|██████████| 954/954 [00:03<00:00, 265.67it/s]
